# Deep Learning for the Titanic dataset

The idea of this notebook is to use and evaluate deep learning models on the Titanic dataset

In [22]:
# import dependencies
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score

%matplotlib inline

In [2]:
# load dataset
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [3]:
# features
print(train.columns.values)

['PassengerId' 'Survived' 'Pclass' 'Name' 'Sex' 'Age' 'SibSp' 'Parch'
 'Ticket' 'Fare' 'Cabin' 'Embarked']


In [4]:
# training examples
train.head()

PassengerId  Survived  Pclass  \
0            1         0       3   
1            2         1       1   
2            3         1       3   
3            4         1       1   
4            5         0       3   

                                                Name     Sex   Age  SibSp  \
0                            Braund, Mr. Owen Harris    male  22.0      1   
1  Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0      1   
2                             Heikkinen, Miss. Laina  female  26.0      0   
3       Futrelle, Mrs. Jacques Heath (Lily May Peel)  female  35.0      1   
4                           Allen, Mr. William Henry    male  35.0      0   

   Parch            Ticket     Fare Cabin Embarked  
0      0         A/5 21171   7.2500   NaN        S  
1      0          PC 17599  71.2833   C85        C  
2      0  STON/O2. 3101282   7.9250   NaN        S  
3      0            113803  53.1000  C123        S  
4      0            373450   8.0500   NaN        S

In [5]:
train.tail()

PassengerId  Survived  Pclass                                      Name  \
886          887         0       2                     Montvila, Rev. Juozas   
887          888         1       1              Graham, Miss. Margaret Edith   
888          889         0       3  Johnston, Miss. Catherine Helen "Carrie"   
889          890         1       1                     Behr, Mr. Karl Howell   
890          891         0       3                       Dooley, Mr. Patrick   

        Sex   Age  SibSp  Parch      Ticket   Fare Cabin Embarked  
886    male  27.0      0      0      211536  13.00   NaN        S  
887  female  19.0      0      0      112053  30.00   B42        S  
888  female   NaN      1      2  W./C. 6607  23.45   NaN        S  
889    male  26.0      0      0      111369  30.00  C148        C  
890    male  32.0      0      0      370376   7.75   NaN        Q

In [6]:
train.info()
print('_'*40)
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 83.6+ KB
________________________________________
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
PassengerId    418 non-null int64
Pclass         418 non-null int64
Name           418 non-null object
Sex            418 non-null object
Age            332 non-null float64
SibSp          418 non-null int64
Parch          418 non-null int64
Ticket         418 non-null

In [7]:
train.describe()

PassengerId    Survived      Pclass         Age       SibSp  \
count   891.000000  891.000000  891.000000  714.000000  891.000000   
mean    446.000000    0.383838    2.308642   29.699118    0.523008   
std     257.353842    0.486592    0.836071   14.526497    1.102743   
min       1.000000    0.000000    1.000000    0.420000    0.000000   
25%     223.500000    0.000000    2.000000   20.125000    0.000000   
50%     446.000000    0.000000    3.000000   28.000000    0.000000   
75%     668.500000    1.000000    3.000000   38.000000    1.000000   
max     891.000000    1.000000    3.000000   80.000000    8.000000   

            Parch        Fare  
count  891.000000  891.000000  
mean     0.381594   32.204208  
std      0.806057   49.693429  
min      0.000000    0.000000  
25%      0.000000    7.910400  
50%      0.000000   14.454200  
75%      0.000000   31.000000  
max      6.000000  512.329200

In [8]:
train.describe(include=['O'])

Name   Sex Ticket Cabin Embarked
count                    891   891    891   204      889
unique                   891     2    681   147        3
top     Stoytcheff, Mr. Ilia  male   1601    G6        S
freq                       1   577      7     4      644

In [13]:
train.drop('Survived', axis=1)

PassengerId  Pclass                                               Name  \
0              1       3                            Braund, Mr. Owen Harris   
1              2       1  Cumings, Mrs. John Bradley (Florence Briggs Th...   
2              3       3                             Heikkinen, Miss. Laina   
3              4       1       Futrelle, Mrs. Jacques Heath (Lily May Peel)   
4              5       3                           Allen, Mr. William Henry   
5              6       3                                   Moran, Mr. James   
6              7       1                            McCarthy, Mr. Timothy J   
7              8       3                     Palsson, Master. Gosta Leonard   
8              9       3  Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)   
9             10       2                Nasser, Mrs. Nicholas (Adele Achem)   
10            11       3                    Sandstrom, Miss. Marguerite Rut   
11            12       1                           Bonnell, Miss. Elizabeth   
12            13       3                     Saundercock, Mr. William Henry   
13            14       3                        Andersson, Mr. Anders Johan   
14            15       3               Vestrom, Miss. Hulda Amanda Adolfina   
15            16       2                   Hewlett, Mrs. (Mary D Kingcome)    
16            17       3                               Rice, Master. Eugene   
17            18       2                       Williams, Mr. Charles Eugene   
18            19       3  Vander Planke, Mrs. Julius (Emelia Maria Vande...   
19            20       3                            Masselmani, Mrs. Fatima   
20            21       2                               Fynney, Mr. Joseph J   
21            22       2                              Beesley, Mr. Lawrence   
22            23       3                        McGowan, Miss. Anna "Annie"   
23            24       1                       Sloper, Mr. William Thompson   
24            25       3                      Palsson, Miss. Torborg Danira   
25            26       3  Asplund, Mrs. Carl Oscar (Selma Augusta Emilia...   
26            27       3                            Emir, Mr. Farred Chehab   
27            28       1                     Fortune, Mr. Charles Alexander   
28            29       3                      O'Dwyer, Miss. Ellen "Nellie"   
29            30       3                                Todoroff, Mr. Lalio   
..           ...     ...                                                ...   
861          862       2                        Giles, Mr. Frederick Edward   
862          863       1  Swift, Mrs. Frederick Joel (Margaret Welles Ba...   
863          864       3                  Sage, Miss. Dorothy Edith "Dolly"   
864          865       2                             Gill, Mr. John William   
865          866       2                           Bystrom, Mrs. (Karolina)   
866          867       2                       Duran y More, Miss. Asuncion   
867          868       1               Roebling, Mr. Washington Augustus II   
868          869       3                        van Melkebeke, Mr. Philemon   
869          870       3                    Johnson, Master. Harold Theodor   
870          871       3                                  Balkic, Mr. Cerin   
871          872       1   Beckwith, Mrs. Richard Leonard (Sallie Monypeny)   
872          873       1                           Carlsson, Mr. Frans Olof   
873          874       3                        Vander Cruyssen, Mr. Victor   
874          875       2              Abelson, Mrs. Samuel (Hannah Wizosky)   
875          876       3                   Najib, Miss. Adele Kiamie "Jane"   
876          877       3                      Gustafsson, Mr. Alfred Ossian   
877          878       3                               Petroff, Mr. Nedelio   
878          879       3                                 Laleff, Mr. Kristo   
879          880       1      Potter, Mrs. Thomas Jr (L

## Split dataset

In [15]:
X_train = train.drop('Survived', axis=1)
Y_train = train['Survived']

In [16]:
print(X_train.shape)
print(Y_train.shape)

(891, 11)
(891,)


## Model

In [17]:
def create_baseline():
    # create model
    model = Sequential()
    model.add(Dense(10, input=11, kernel_initializer='normal', activation='relu'))
    model.add(Dense(1, kernel_initializer='normal', activation='sigmoid'))
    # compile model using logarithmic loss function and the adam optimizer
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [23]:
estimator = KerasClassifier(build_fn=create_baseline, nb_epoch=1, batch_size=5, verbose=0)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
results = cross_val_score(estimator, X_train, Y_train, cv=kfold)
print("Results: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

TypeError: ('Keyword argument not understood:', 'input')